## 2. Preparação dos dados

* **Data Cleansing**: Identificar e corrigir inconsistências, valores em falta ou inválidos.
* **Data Transformation**: Transformar os dados, se necessário, para uniformizar as variáveis.
* **Data Imputation**: Preencher valores em falta com métodos apropriados.
* **Data Weighting and Balancing**: Garantir que os dados estão equilibrados, especialmente em casos de classes desiguais.
* **Data Filtering**: Selecionar os dados mais relevantes.
* **Data Reduction**: Reduzir o volume total de dados, mantendo informação essencial.
* **Data Sampling (Records)**: Obter uma amostra representativa dos registos.
* **Dimensionality Reduction (Variables)**: Reduzir o número de variáveis, mantendo as mais significativas.
* **Data Discretization (Values)**: Agrupar valores contínuos em categorias.
* **Data Derivation**: Criar novas variáveis com base nas existentes.

--- 

## 2.1 - Bibliotecas Utilizadas

In [26]:
# Importar Bibliotecas

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib


## 2.2 - Dataset Original
https://www.kaggle.com/datasets/kumarajarshi/life-expectancy-who

In [ ]:
# Ler o arquivo CSV "Life Expectancy Data.csv" guaradado na pasta data e carregar o ficheiro para um DataFrame designado de "df_original".
df_original = pd.read_csv('./data/Life Expectancy Data.csv')

# 2.3 - Informações Gerais Dataset

- Primeiras linhas do dataset
- Principais informações do dataset
- Estatísticas do dataset

In [ ]:
df_original.head()

In [ ]:
df_original.info()

In [ ]:
df_original.describe().round(2)

# 2.4 - Preparaçao Dataset

- Limpar os nomes das colunas
- Verificar se existem valores nulos
- Tratar valores ausentes ou nulos
- Confirmar se temos valores duplicados
- Confirmar quais valores únicos de cada variável

In [ ]:
df_original.columns = df_original.columns.str.strip()

In [ ]:
df_original.isnull().sum()

In [ ]:
# Tratar valores ausentes para 'GDP' e 'Schooling'
df_original['GDP'] = df_original['GDP'].fillna(df_original['GDP'].median())
df_original['Schooling'] = df_original['Schooling'].fillna(df_original['Schooling'].median())
df_original.isnull().sum()

In [ ]:
df_original.duplicated().sum()

In [ ]:
initial_counts = df_original["Country"].value_counts()
print("Initial counts of target variable classes:")
print(initial_counts)

# 2.5 - Ananálise exploratória

1. Estatísticas descritivas: Resumo estatístico para variáveis numéricas.
2. Distribuição das variáveis: Identificação de padrões ou possíveis outliers.
3. Correlação entre variáveis: Análise da relação entre elas, para detectar redundâncias ou dependências.


In [ ]:
# Função para categorizar as variáveis de um DataFrame
def analyze_variable_types(df):
    variable_types = {
        "binary": [],  # Variáveis binárias (0 ou 1)
        "categorical": [],  # Variáveis categóricas (discretas)
        "continuous": [],  # Variáveis contínuas (numéricas)
    }

    # Analisar cada coluna no DataFrame
    for column in df.columns:
        unique_values = df[column].dropna().unique()  # Eliminar valores nulos

        if set(unique_values).issubset({0, 1, 0.0, 1.0}):
            variable_types["binary"].append(column)
        elif df[column].dtype in ["int64", "float64"] and len(unique_values) > 3:
            variable_types["continuous"].append(column)
        else:
            variable_types["categorical"].append(column)

    # Output do número de variáveis por tipo
    print(f"\nNúmero de variáveis binárias: {len(variable_types['binary'])}")
    print(f"Número de variáveis categóricas: {len(variable_types['categorical'])}")
    print(f"Número de variáveis contínuas: {len(variable_types['continuous'])}")

    return variable_types


# Aplicar a função ao conjunto de dados
variable_types = analyze_variable_types(df_original)

# Exibir os tipos de variáveis
print("\nVariáveis Binárias:")
print(variable_types["binary"])
print("\nVariáveis Categóricas:")
print(variable_types["categorical"])
print("\nVariáveis Contínuas:")
print(variable_types["continuous"])


In [ ]:
# Transformar a variável 'Status' em valores numéricos (1 e 2)
df_original['Status'] = df_original['Status'].map({'Developed': 1, 'Developing': 2})

# Verificar as transformações realizadas
df_original['Status'].head()


In [ ]:
df_original.info()

In [ ]:
# Verificar os valores únicos na coluna 'Status'
unique_status_values = df_original['Status'].unique()
unique_status_values


# 2.6 - Normalização e Variáveis Mais Importantes

- Definir o alvo (y) e as características (X)
- Normalizar as variáveis contínuas com MinMaxScaler
- Obter importâncias das variáveis


In [ ]:
# Alvo (expectativa de vida)
y = df_original['Life expectancy']

# Características socioeconómicas selecionadas
socioeconomic_features = ['GDP', 'Schooling', 'Income composition of resources', 'Alcohol', 'Population']
X = df_original[socioeconomic_features]


In [ ]:

# Normalizar as variáveis contínuas com MinMaxScaler
scaler = MinMaxScaler()
X_scaled = X.copy()

# Aplicar a normalização
X_scaled[socioeconomic_features] = scaler.fit_transform(X[socioeconomic_features])

# Guardar o scaler para uso futuro
joblib.dump(scaler, "./data/scaler.pkl")

# Confirmar a normalização
X_scaled.head()


In [ ]:

# Limpar dados: Remover colunas não numéricas e valores nulos
data_cleaned = df_original.drop(['Country', 'Year', 'Status'], axis=1).dropna()

# Separar variáveis dependentes e independentes
X = data_cleaned.drop('Life expectancy', axis=1)
y = data_cleaned['Life expectancy']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criar o modelo Random Forest
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Obter importâncias das variáveis
importances = model.feature_importances_

# Criar DataFrame para visualizar as importâncias
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

# Criar um gráfico de barras para importância das variáveis
plt.figure(figsize=(10, 6))
plt.bar(feature_importance['Feature'], feature_importance['Importance'], edgecolor='black')
plt.xticks(rotation=90)
plt.xlabel('Variáveis')
plt.ylabel('Importância')
plt.title('Importância das Variáveis para a Esperança Média de Vida')
plt.tight_layout()

# Exibir o gráfico
plt.show()



# 2.7 - Preparação dos dados de treino e teste

In [27]:
# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar os dados
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
# Converter os arrays em DataFrames ou Series
X_train_df = pd.DataFrame(X_train, columns=X.columns)
X_test_df = pd.DataFrame(X_test, columns=X.columns)
y_train_df = pd.Series(y_train, name='Life expectancy')
y_test_df = pd.Series(y_test, name='Life expectancy')

# Guardar os dados em ficheiros CSV
X_train_df.to_csv('./data/X_train.csv', index=False)
X_test_df.to_csv('./data/X_test.csv', index=False)
y_train_df.to_csv('./data/y_train.csv', index=False)
y_test_df.to_csv('./data/y_test.csv', index=False)
